In [1]:
import sys
import os
from typing import Tuple

from numpy.random.mtrand import random

sys.path.append('/home/danillorp/Área de Trabalho/github/fema/src/')

import numpy as np
import math 

from fem_basis import Basis 

import pandas
import fema_classifier
import fema_feature_learning
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn import preprocessing

DEBUG = False


/tmp/ipykernel_117772/311831510.py:14: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas


In [5]:
ml_datasets = ['vehicle','credit-g','satellite']

#'airlines','adult','SpeedDating','blood-transfusion-service-center','mnist_784','one-hundred-plants-texture','steel-plates-fault',
#'kr-vs-kp','arrhythmia','bank-marketing','PhishingWebsites','kc1','pc1','cmc','mfeat-factors','KDDCup09_appetency'
print('The experiment will run for ',len(ml_datasets),' datasets')        

The experiment will run for  3  datasets


In [6]:
def experiment(dataset_name:str, data_x:pandas.core.frame.DataFrame, data_y:pandas.core.series.Series, test_size:float, n_runs:int, DEBUG:bool=False):
    for i in range(n_runs):
        #split datasets
        train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, test_size=test_size)
        eval_x, test_x, eval_y, test_y = train_test_split(test_x, test_y, test_size=test_size)

    
        if DEBUG:
            print('**********',dataset_name,'**********')
            print('train_size',len(train_x),'classes:',set(train_y))
            print('test_size',len(test_x),'classes:',set(test_y))
        
        #reorganize the arrays to be an input of FEMa library
        train_y = train_y[:,np.newaxis]
        test_y = test_y[:,np.newaxis]
        eval_y = eval_y[:,np.newaxis]
        
        #Apply data scaling based on training data
        scaler = StandardScaler()
    
        train_x = scaler.fit_transform(train_x)
        test_x = scaler.transform(test_x)
        eval_x = scaler.transform(eval_x)
        
        #Run the model without the SemiSupervised
        model_fl = fema_feature_learning.FEMaFeatureLearning(k=10,basis=fema_classifier.Basis.radialBasis)
        features_weigths = model_fl.fit(train_x, train_y, eval_x, eval_y)


        model_fema_original = fema_classifier.FEMaClassifier(k=10,basis=fema_classifier.Basis.radialBasis)
        model_fema_original.fit(train_x,train_y)

        pred, confidence_level = model_fema_original.predict(test_x,10)

        cm_fema_original = confusion_matrix(test_y,pred)
        bal_original = balanced_accuracy_score(test_y, pred)
        acc_original = accuracy_score(test_y, pred)

        features_qtd = train_x.shape[1]
        for c in range(len(set(train_y[:,0]))):
            train_x_cp = train_x.copy()
            test_x_cp  = test_x.copy()
            
        
            for f in range(features_qtd):
                mask_inter = train_y[:,0] != c
                mask_intra = train_y == c
                
                train_x_cp[:,f] =   (train_x_cp[:,f])*  abs(features_weigths[model_fl.INTER, c, f]/features_weigths[model_fl.INTRA, c, f])
                test_x_cp[:,f] =    (test_x_cp[:,f]) * abs(features_weigths[model_fl.INTER, c, f]/features_weigths[model_fl.INTRA, c, f])
                
        
        model_fema_adjusted = fema_classifier.FEMaClassifier(k=10,basis=fema_classifier.Basis.radialBasis)
        model_fema_adjusted.fit(train_x_cp,train_y)
    
        pred, confidence_level = model_fema_adjusted.predict(test_x_cp,10)
    
        cm_fema_adj = confusion_matrix(test_y,pred)
        bal_adj = balanced_accuracy_score(test_y, pred)
        acc_adj = accuracy_score(test_y, pred)

        print('********************',dataset_name,'********************')
        print('CM Original:',cm_fema_original)
        print('CM Adjusted:',cm_fema_adj)
        
        print('BalAcc Original:',bal_original)
        print('BalAcc Adjusted:',bal_adj)

        print('Acc Original:',acc_original)
        print('Acc Adjusted:',acc_adj)

        
        return


In [7]:
for dataset in ml_datasets: 
    data = fetch_openml(dataset,version=1)
    X, y = data['data'], data['target']

    X = X.select_dtypes(include=np.number)

    le = preprocessing.LabelEncoder()
    le.fit(y)
    y = le.transform(y)

    if DEBUG:
        print(dataset, X.shape, y.shape, type(X), type(y))
        print(X.dtypes)
    
    experiment(dataset_name=dataset, data_x=X, data_y=y, test_size=0.90,n_runs=3,DEBUG=True)

********** vehicle **********
train_size 84 classes: {0, 1, 2, 3}
test_size 686 classes: {0, 1, 2, 3}
******************** vehicle ********************
CM Original: [[175   4   3   3]
 [ 18  73  67  18]
 [ 22  63  70  16]
 [ 19  10   6 119]]
CM Adjusted: [[179   1   0   5]
 [133  18  24   1]
 [112  25  31   3]
 [ 84   2   3  65]]
BalAcc Original: 0.6357006677730361
BalAcc Adjusted: 0.4183011916564548
Acc Original: 0.6370262390670554
Acc Adjusted: 0.4271137026239067
********** credit-g **********
train_size 100 classes: {0, 1}
test_size 810 classes: {0, 1}
******************** credit-g ********************
CM Original: [[104 142]
 [148 416]]
CM Adjusted: [[111 135]
 [160 404]]
BalAcc Original: 0.580176440062273
BalAcc Adjusted: 0.5837657844663553
Acc Original: 0.6419753086419753
Acc Adjusted: 0.6358024691358025
********** satellite **********
train_size 510 classes: {0, 1}
test_size 4131 classes: {0, 1}
******************** satellite ********************
CM Original: [[  53    6]
 [ 297